# **Next Word Predictor**

## I have implemented a 'next word predictor' using 2 different methods.





1.    **Using GloVe pretrained Embedding Layer**
  - I initially tried an embedding layer which was trained from scratch along with the model, but i did not get a good result. So I have chosen a pretrained embedding layer because of the low resource available and the fact that the corpus has more High Frequency words and long sentences.
So it would be better to start with pretrained weights as it would be easier to map High frequency words.



2.   **Tried adding POS_Tag as a feature to the input and feeding directly to LSTM**
        - I had not  come across any implementations of using POS( Part of Speech) of words as an additional feature, so i have given it a try. It does not work as espected though.







---


# **Implementation 1**

---




In [0]:
!git clone https://github.com/svtsanoj/Word-Predictor.git

!7z e "/content/Word-Predictor/Word-Predictor.7z" 

fatal: destination path 'Word-Predictor' already exists and is not an empty directory.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/Word-Predictor/                                  1 file, 71604256 bytes (69 MiB)

Extracting archive: /content/Word-Predictor/Word-Predictor.7z
--
Path = /content/Word-Predictor/Word-Predictor.7z
Type = 7z
Physical Size = 71604256
Headers Size = 273
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      6% 1 - glove.6B.50d.txt                          13% 1 - glove.6B.50d.txt                          19% 1 - glove.6B.50d.txt                         

In [0]:
import tensorflow as tf
import re
import keras
from numpy import array, asarray
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras import models
import numpy as np
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


##Validation Text
This is a random validation text that i wrote making sure most words are in vocabulary of the corpus.

Validation Text is added to the corpus and later split as training and testing data to ease preprocessing efforts.

In [0]:
val_text= """
the man pushed the woman away and asked the surgeon whether the man is dead. 
How can someone do this to him? 
he was a good man. 
I wanted him to die, said the woman. 
the man was agitated. 
He turned around as there was a hand on his shoulder.
A tall man stood motionless. 
I should not go out at this hour of the day. 
It was cold outside and I was wet. 
I looked up to see if the stars were out. 
I do not know what to say. 
This information is useless. 
this produced a good impression on my mother. 
I will never leave you alone or out of sight. 
you must be with me.
where was that. 
where is it"""

## Preprocessing Data
**Split Into Sentences** : If the words from the corpus are provided as a continous sequence to train on, It loses its syntactic structure when the end of a certain sentences is combined with the beginning of another.

In [0]:
path="/content/corpus.txt"
with open(path) as f:
    text = f.read().lower().replace("\n"," ") # there are no \t so we just replace \n
    
text = re.split("[.?!;]+", text)              
val_text = re.split("[.?!;]+", val_text) 
val_len = len(val_text)
i = 0
dummy = []
for sentence in text:
    text[i] = re.sub("\s+", " ", (re.sub("[^a-z ]+", " ", sentence))) # replace everything but lower case letters and spaces with " " and remove extra spaces.
    k = text[i].split()
    if(len(k)>1):
      dummy.append(k)
    i += 1
text = dummy

print("Total Sentences in Training Data: " ,len(text))
print("Total Sentences in Validation Data: ", val_len)
text[1] 

Total Sentences in Training Data:  256
Total Sentences in Validation Data:  17


['the', 'night', 'was', 'wet', 'and', 'cold']

In [0]:
s = 0
for i in range(len(text)):
  l = len(text[i])
  # print(l)
  s += l
avg=s/len(text)
print("avg sentence length: %f"%(avg))

avg sentence length: 17.925781


## Keras tokenizer
### Alternatively, mapped words to tokens
  ```
word_to_ix = { word:i for i,word in enumerate(wh_text) } #From Andrej Karpathy's Char RNN Implementation
seqlength = 5
X,y=[],[]
#wrote this for having all words as a continous sequence and splitting Input sequence with length = 5
for i, word in enumerate(wh_text[:-seqlength]): 
  X.append(list(word_to_ix[wrd] for wrd in wh_text[i:i+seqlength]))
  y.append(word_to_ix[wh_text[i+seqlength]])
print(X[:5])
print(y[:5])
  ```
 ### But keras tokenizes based on the frequency of words. Which is an useful feature.

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
data = tokenizer.texts_to_sequences(text)
vocab_size = len(tokenizer.word_counts) + 1
print("vocab_size", vocab_size)

vocab_size 1335


## Training and Validation Data

As we have to work with limited data and that too restricting ourselves from picking word combinations only from each sentences, I have implemented a **n-gram** split of sentences.

###Eg: a man was standing outside the house. 
So let's split these words into a sequences of Maximum length of 5 and Minimum of 1 for input (X) and , 1 word for output (y).


**len = 5**

**X:**["a", "man", "was", "standing", "outside"],  **y:**[the]

**X:**["man", "was", "standing", "outside", "the"],  **y:**[house]

**len =2**

**X:**[0, 0,  0, "a", "man"],     **y:**["was"]

**X:**[0, 0, 0, "man", "was"],     **y:**["standing"]

.

.

.


In [0]:
seqlength = 8
X, y, X_val, y_val = [], [], [], []
for i in range(1, seqlength+1):  #1-5 is the different sequence length of the input data
  iter=0
  for j in range(len(data)):
    for k in range(len(data[iter])-i):
      if(j<len(data)-val_len):
        X.append(data[iter][k : k+i])
        X[-1] = [0]*(seqlength-i) + X[-1] #pre-padding
        #print("i j k data[iter][k:k+i] X", i,j,k,data[iter][k:k+i], X)
        y.append( data[iter][k+i] )
      else:
        X_val.append(data[iter][k : k+i])
        X_val[-1] = [0]*(seqlength-i) + X_val[-1] #pre-padding
        #print("i j k data[iter][k:k+i] X", i,j,k,data[iter][k:k+i], X)
        y_val.append( data[iter][k+i] )
    iter += 1

## Convert to Arrays
 X and y are converted into array data type from being lists. 
 
 Here y is made into one shot encoded array using the keras to_cateorical() function.

 

In [0]:
X = np.array(X, dtype='int32')
y = to_categorical(y, num_classes=vocab_size)
X_val = np.array(X_val, dtype='int32')
y_val = to_categorical(y_val, num_classes=vocab_size)

print(X.shape)
print(y.shape)

(27644, 8)
(27644, 1335)


## GloVe Embedding Layer
Have to download the 50 Dimensional GloVe Embeddings which is arounnd 160MB. It represents 400000 words as 50D vectors.

[Reference for loading and using Pretrained Glove embedding](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/)

In [0]:
# Used From GloVe REFERENCE

embeddings_index = dict()
f = open('/content/glove.6B.50d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:])
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
# Used From GloVe REFERENCE

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 50))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

## Model

the pretrained embedding values are loaded into the embedding layer. We can choose to either update these weights or not using trainable = True or False

Itereated over different parameters:
- LSTM layer with 50 Units each
- Dense layer neurons between 100 to 400
- Batch Sizes - None, 128, 256

- **Sequence Length** : 3 , 5, 8

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=seqlength, trainable=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(400,activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, batch_size=256, validation_data=(X_val, y_val), verbose=1, shuffle='True')

model.save("word_model_8_gram.h5")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 8, 50)             66800     
_________________________________________________________________
lstm_7 (LSTM)                (None, 8, 100)            60400     
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 400)               40400     
_________________________________________________________________
dense_6 (Dense)              (None, 1336)              535736    
Total params: 783,736
Trainable params: 783,736
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 27644 samples, validate on 450 samples
Epoch 1/100
27644/27644 [==============================] - 5s 163us/step - loss: 6.2021 - accuracy: 0.0695 - val_loss: 5.8564 - val_accuracy: 0.0911
Epoch 2/100
27644/27644 [==============================] - 4s 150us/step - loss: 5.8735 - accuracy: 0.0729 - val_loss: 5.6522 - val_accuracy: 0.1000
Epoch 3/100
27644/27644 [==============================] - 4s 147us/step - loss: 5.4246 - accuracy: 0.0966 - val_loss: 5.4341 - val_accuracy: 0.1222
Epoch 4/100
27644/27644 [==============================] - 4s 142us/step - loss: 4.7994 - accuracy: 0.1219 - val_loss: 5.6140 - val_accuracy: 0.0978
Epoch 5/100
27644/27644 [==============================] - 4s 140us/step - loss: 4.3176 - accuracy: 0.1378 - val_loss: 6.0479 - val_accuracy: 0.1089
Epoch 6/100
27644/27644 [==============================] - 4s 140us/step - loss: 3.8836 - accuracy: 0.1689 - val_loss: 6.3224 - val_accuracy: 0.1111
Epoch 7/100
27644/27644 [==============================] -

## Predictions:
I have modified and used used the prediction code from this [Reference](https://towardsdatascience.com/exploring-the-next-word-predictor-5e22aeb85d8f) for padding and extracting maximum probable next word from the output vector using argsort().

In [0]:
model_glove=tf.keras.models.load_model('/content/word_model_8_gram.h5')

from keras.preprocessing.sequence import pad_sequences

def predict_glove(input):                       
  input_text = re.sub("\s+", " ", (re.sub("[^a-z ]+", " ", input)))
  encoded_text = tokenizer.texts_to_sequences([input_text])[0]

  pad_encoded = pad_sequences([encoded_text], maxlen=seqlength, truncating='pre') # code from REFERENCE {
  print(encoded_text, pad_encoded)
  i = (model_glove.predict(pad_encoded)[0]).argsort()[-1:][::-1][0]
  pred_word = tokenizer.index_word[i]
  print("Next word:",pred_word)                                                   # }


In [0]:
predict_glove("the dead body was in")

[1, 149, 108, 7, 6] [[  0   0   0   1 149 108   7   6]]
Next word: spite


In [0]:
predict_glove(" the man and woman began to wonder whether") # similar to corpus : then he began to wonder when

[1, 28, 2, 24, 502, 4, 503, 295] [[  1  28   2  24 502   4 503 295]]
Next word: i


In [0]:
predict_glove("the dead body was in the dark room and there was a") 

[1, 149, 108, 7, 6, 1, 91, 2, 45, 7, 5] [[ 7  6  1 91  2 45  7  5]]
Next word: nervous




---





---


#**Implementation 2**


---
Similar to the above implementation but because i am appending the POS_tag feature to the input sentence, it makes the 1D vector into 2D vector, so i will be directly inputing this to an LSTM Layer omitting the Embedding Layer.


In [0]:
import tensorflow as tf
import re
import keras
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras import models
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import nltk
from nltk import word_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##Validation Text

In [0]:
val_text= """
the man pushed the woman away and asked the surgeon whether the man is dead. 
How can someone do this to him? 
he was a good man. 
I wanted him to die, said the woman. 
the man was agitated. 
He turned around as there was a hand on his shoulder.
A tall man stood motionless. 
I should not go out at this hour of the day. 
It was cold outside and I was wet. 
I looked up to see if the stars were out. 
I do not know what to say. 
This information is useless. 
this produced a good impression on my mother. 
I will never leave you alone or out of sight. 
you must be with me.
where was that. 
where is it"""

##Data Preprocessing

I've written the processes as functions, so that i can call them in the prediction function as it can not use keras paddding features.

In [0]:
path="/content/corpus.txt"
with open(path) as f:
    text = f.read().lower().replace("\n"," ")


def preprocess(text, val_text = val_text):   
  text = re.split("[.?!;]+", text)        # split the text into sentences
  val_text = re.split("[.?!;]+", val_text)
  i = 0
  dummy = []
  for sentence in text:
    text[i] = re.sub("[^a-z ]+", "", sentence)
    k = text[i].split()                     # split sentences into words
    if(len(k)>1):                           # omit sentences that have only one word
      dummy.append(k)
    i += 1
  text = dummy
  return text, val_text

text, val_text = preprocess(text)
val_len = len(val_text)
text[-17:]                                 # Validation texts

[['the',
  'man',
  'pushed',
  'the',
  'woman',
  'away',
  'and',
  'asked',
  'the',
  'surgeon',
  'whether',
  'the',
  'man',
  'is',
  'dead'],
 ['how', 'can', 'someone', 'do', 'this', 'to', 'him'],
 ['he', 'was', 'a', 'good', 'man'],
 ['i', 'wanted', 'him', 'to', 'die', 'said', 'the', 'woman'],
 ['the', 'man', 'was', 'agitated'],
 ['he',
  'turned',
  'around',
  'as',
  'there',
  'was',
  'a',
  'hand',
  'on',
  'his',
  'shoulder'],
 ['a', 'tall', 'man', 'stood', 'motionless'],
 ['i', 'should', 'not', 'go', 'out', 'at', 'this', 'hour', 'of', 'the', 'day'],
 ['it', 'was', 'cold', 'outside', 'and', 'i', 'was', 'wet'],
 ['i', 'looked', 'up', 'to', 'see', 'if', 'the', 'stars', 'were', 'out'],
 ['i', 'do', 'not', 'know', 'what', 'to', 'say'],
 ['this', 'information', 'is', 'useless'],
 ['this', 'produced', 'a', 'good', 'impression', 'on', 'my', 'mother'],
 ['i', 'will', 'never', 'leave', 'you', 'alone', 'or', 'out', 'of', 'sight'],
 ['you', 'must', 'be', 'with', 'me'],
 ['where

##Annotatate text with POS_tag

nltk's pos tagger library is used to annotate all the words in the text.

I also maintain a dictionary of all the pos_taggers to append them as features in the next step

In [0]:
def text_to_pos(text):
  dummy, pos_text=[], []
  pos_dict = {}
  iter = 1
  for i in range(len(text)):
    dummy=[]
    for word, pos in nltk.pos_tag(text[i]):
      if(pos not in pos_dict.keys()):         
        pos_dict[pos] = iter                # make a dictionary for POS
        iter+=1
      dummy.append([word] + [pos])
    pos_text.append(dummy)

  return pos_text, pos_dict

pos_text , pos_dict = text_to_pos(text)
print(pos_text[0:2])
print(pos_dict)

[[['one', 'CD'], ['winters', 'NNS'], ['evening', 'VBG'], ['towards', 'IN'], ['the', 'DT'], ['close', 'NN'], ['of', 'IN'], ['the', 'DT'], ['year', 'NN'], ['or', 'CC'], ['within', 'IN'], ['a', 'DT'], ['year', 'NN'], ['or', 'CC'], ['two', 'CD'], ['of', 'IN'], ['that', 'DT'], ['time', 'NN'], ['a', 'DT'], ['young', 'JJ'], ['medical', 'JJ'], ['practitioner', 'NN'], ['recently', 'RB'], ['established', 'VBN'], ['in', 'IN'], ['business', 'NN'], ['was', 'VBD'], ['seated', 'VBN'], ['by', 'IN'], ['a', 'DT'], ['cheerful', 'JJ'], ['fire', 'NN'], ['in', 'IN'], ['his', 'PRP$'], ['little', 'JJ'], ['parlour', 'JJ'], ['listening', 'NN'], ['to', 'TO'], ['the', 'DT'], ['wind', 'NN'], ['which', 'WDT'], ['was', 'VBD'], ['beating', 'VBG'], ['the', 'DT'], ['rain', 'NN'], ['in', 'IN'], ['pattering', 'VBG'], ['drops', 'NNS'], ['against', 'IN'], ['the', 'DT'], ['window', 'NN'], ['or', 'CC'], ['rumbling', 'VBG'], ['dismally', 'RB'], ['in', 'IN'], ['the', 'DT'], ['chimney', 'NN']], [['the', 'DT'], ['night', 'NN'], 

##Keras Tokenizer

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
data = tokenizer.texts_to_sequences(text)
vocab_size = len(tokenizer.word_counts) + 1
print(vocab_size)

1345


##Vectorised [Word, POS_Tag]
Now, combining both the Keras tokenised words and the corresponding integer for the POS_tag whic was stored in the dictionary (pos_dict) we otain our dataset which is appended with POS feature

In [0]:
def text_to_vector(pos_text):
  dummy, pos_vector = [], []

  for i in range(len(pos_text)):
    dummy=[]
    for word, pos in pos_text[i]:
      dummy.append( tokenizer.texts_to_sequences([word])[0] + [pos_dict[pos]] )
    pos_vector.append(dummy)                      # [ Token(word) + Token(POS) ]
  return pos_vector
pos_vector = text_to_vector(pos_text)
print(pos_vector[1:3])
print(len(data[1]))

[[[1, 5], [77, 6], [7, 11], [199, 8], [2, 7], [78, 8]], [[8, 15], [15, 11], [34, 10], [467, 3], [54, 4], [200, 6], [2, 7], [119, 6], [1, 5], [278, 8], [120, 6], [2, 7], [7, 11], [49, 9], [279, 9], [468, 3], [6, 4], [9, 12], [469, 6], [2, 7], [470, 2], [92, 16], [64, 4], [201, 8], [471, 8], [2, 7], [280, 16], [64, 4], [201, 8], [472, 8], [473, 3], [5, 5], [474, 6], [475, 2], [6, 4], [9, 12], [476, 6], [281, 6]]]
6


##Train Validation Split

The POS_tagged sentences are combined as shown previosly. They are made into 8_gram sequences so each sequence in the dataset is in the shape of (seqlength, 2) which is (8, 2)

In [0]:
seqlength = 8
X, y, X_val, y_val = [], [], [], []
for i in range(1, seqlength+1):  #1-5 is the different sequence length of the input data
  iter=0
  for j in range(len(pos_vector)):
    for k in range(len(pos_text[iter])-i):
      if(j<len(pos_vector)-val_len):
        X.append(pos_vector[iter][k : k+i])
        X[-1] = [[0, 0]]*(seqlength-i) + X[-1]        #pre-padding
        #print("i j k data[iter][k:k+i] X", i,j,k,data[iter][k:k+i], X)
        y.append( pos_vector[iter][k+i][1] )
      else:
        X_val.append(pos_vector[iter][k : k+i])
        X_val[-1] = [[0,0]]*(seqlength-i) + X_val[-1] #pre-padding
        #print("i j k data[iter][k:k+i] X", i,j,k,data[iter][k:k+i], X)
        y_val.append( pos_vector[iter][k+i][1] )
    iter += 1

X = np.array(X, dtype='int32')
y = to_categorical(y, num_classes=vocab_size)
X_val = np.array(X_val, dtype='int32')
y_val = to_categorical(y_val, num_classes=vocab_size)

print(X.shape)
print(y.shape)

(27219, 8, 2)
(27219, 1345)


##Model

Iterated over different, 
- Number of LSTM layers
- Number of units and nodes

In [0]:
model = Sequential()
model.add(LSTM(100, input_shape=X[0].shape, return_sequences=True)) #return_sequences will be True when we have the next layer as LSTM as well.
model.add(LSTM(100, return_sequences = False))
model.add(Dense(300,activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, validation_data=(X_val, y_val), verbose=1, shuffle='True')

model.save("word_with_pos.h5")

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 8, 100)            41200     
_________________________________________________________________
lstm_17 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_17 (Dense)             (None, 300)               30300     
_________________________________________________________________
dense_18 (Dense)             (None, 1345)              404845    
Total params: 556,745
Trainable params: 556,745
Non-trainable params: 0
_________________________________________________________________
None
Train on 27219 samples, validate on 450 samples
Epoch 1/100
27219/27219 [==============================] - 29s 1ms/step - loss: 2.6001 - accuracy: 0.2503 - val_loss: 2.4462 - val_accuracy: 0.2556
Epoch 2/100
27219/27219 [===================

## Prediction

Used the padding and preprocessing functions i wrote and used argsort() from the [Reference](https://towardsdatascience.com/exploring-the-next-word-predictor-5e22aeb85d8f) as mentioned earlier

In [0]:
from keras.preprocessing.sequence import pad_sequences

model_pos = tf.keras.models.load_model("/content/word_with_pos.h5")

def predict_with_pos(input):
  input_text = input.strip().lower()
  input_text, _ = preprocess(input_text)
  test_pos_text, _ = text_to_pos(input_text)
  encoded_text = text_to_vector(test_pos_text)[0]
  print(encoded_text)
  
  pad_encoded = encoded_text  #for retaining shape
  l_encoded = len(encoded_text)
  
  if(seqlength>=l_encoded):
    pad_encoded = [[[0, 0]]*(seqlength-l_encoded) + encoded_text]
  else:
    pad_ecoded[i] = [encoded_text[-seqlength:]]
  pad_encoded = array(pad_encoded)
  print(pad_encoded)

  i = (model_pos.predict(pad_encoded)[0]).argsort()[-1:][::-1][0]  #from REFERENCE
  pred_word = tokenizer.index_word[i]                          #from REFERENCE
  print("Next word :",pred_word)

In [0]:
predict_with_pos("he thought the wind")

[[8, 15], [121, 11], [1, 5], [275, 6]]
[[[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  8  15]
  [121  11]
  [  1   5]
  [275   6]]]
Next word : was


In [0]:
predict_with_pos("if he could only tell her")

[[36, 4], [8, 15], [80, 18], [127, 9], [203, 19], [19, 12]]
[[[  0   0]
  [  0   0]
  [ 36   4]
  [  8  15]
  [ 80  18]
  [127   9]
  [203  19]
  [ 19  12]]]
Next word : to


In [0]:
predict_with_pos("not a moment is to be")

[[18, 9], [5, 5], [122, 6], [32, 24], [4, 13], [22, 19]]
[[[  0   0]
  [  0   0]
  [ 18   9]
  [  5   5]
  [122   6]
  [ 32  24]
  [  4  13]
  [ 22  19]]]
Next word : it




---



---


#Inference

- The **1st Implementation** which uses a pretrained glove embedding works better and fits better on the corpus.

- **2nd Implementation** mostly Outputs the words that are used frequently in the corpus like [the, of, a, in ,be, that...], I am still learning on how to add features to the Input. Although Validation accuracy is higher for this implementation

- **Validation accuracy** is not a good metric for evaluating our model as the output is categorical and it punishes even similar words that have been predicted as output. 

- An Interesting **[Research paper](https://arxiv.org/pdf/1602.06291.pdf)** that helped me understand the addition of Features in the text is CLSTM ( Contextual LSTM ), where the input text is added with a 'Topic' that is calculated seperately. This Topic feature is not added directly to the input text, rather to the embedding layer.

So i still have to explore how to **add regualrized features to the embedding layer.** There is an [Open Source Code](https://github.com/kafkasl/contextualLSTM) by Pol Alvarez trying to replicate this paper.

[All References](https://github.com/svtsanoj/Word-Predictor/blob/master/references.txt)

--Sanoj